In [1]:
%reload_ext tensorboard

In [17]:
import numpy as np
import tensorflow as tf
import datetime
import tensorboard
tensorboard.__version__

'2.2.1'

In [18]:
def code_length_of(nodes):
    return (nodes * (nodes - 1)) // 2

In [19]:
from  keras.layers import  Conv2D, BatchNormalization, Activation, Add, Flatten, Dense, Dropout, MaxPool2D, AveragePooling2D
import keras.models

In [20]:
def conv_layer(X, num_f, k_size):
    node = X
    node = Conv2D(filters=num_f, kernel_size=k_size,
                  strides=(1, 1), padding='same')(node)
    node = BatchNormalization(axis=3)(node)
    node = Activation('relu')(node)
    return node

In [21]:
def do_something(X, code, num_nodes, num_f, k_size):
    a0 = conv_layer(X, num_f, k_size)
    nodes = {'a1': conv_layer(a0, num_f, k_size)}

    for node in range(2, num_nodes + 1):
        nodes['a' + str(node)] = a0
        end_idx = code_length_of(node)
        start_idx = end_idx - (node - 1)
        prev_nodes = code[start_idx: end_idx]
        connected_nodes = np.where(prev_nodes == 1)[0] + 1  # increment index number
        for prev_node in connected_nodes:
            if (prev_node == node-1):
                nodes['a' + str(node)] = conv_layer(nodes['a' + str(node - 1)],
                                                    num_f, k_size)
            else:
                nodes['a' + str(node)] = Add()([nodes['a' + str(node)],
                                                nodes['a' + str(prev_node)]])

    # Get node last
    node_L = conv_layer(nodes['a' + str(num_nodes)], num_f, k_size)
    return node_L

In [22]:
from keras.layers import Input

In [23]:
def genetic_model(architecture, hyper_params,
                  input_shape=(32, 32, 3), classes=1):
    Pool = MaxPool2D if hyper_params['pooling'] == 'max' else AveragePooling2D

    X_input = Input(input_shape)
    X = X_input

    stages = architecture['stages']
    stage_name, nodes = stages[0]
    end_idx = code_length_of(nodes)
    code = architecture['code'][:end_idx]
    X = do_something(X, code, nodes, hyper_params['filters'],
                     hyper_params['kernel size'])
    X = Pool(pool_size=hyper_params['pool size'],
                 strides=hyper_params['strides'], padding='valid')(X)
    for i in range(1, len(stages)):
        stage_name, nodes = stages[i]
        start_idx = code_length_of(stages[i - 1][1])
        end_idx = start_idx + code_length_of(nodes)
        code = architecture['code'][start_idx: end_idx]
        X = do_something(X, code, nodes, hyper_params['filters'] * (i + 1),
                         hyper_params['kernel size'])
        X = Pool(pool_size=hyper_params['pool size'],
                 strides=hyper_params['strides'], padding='valid')(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(hyper_params['fc units'], activation='relu')(X)
    X = Dropout(hyper_params['drop out'])(X)
    X = Dense(classes, activation='softmax')(X)

    # Create model
    model = keras.models.Model(inputs=X_input, outputs=X)
    model.compile(loss='categorical_crossentropy',
                  optimizer=hyper_params['optimizer'], metrics=['accuracy'])
    return model

In [24]:
import timeit
from keras.datasets import cifar10
from keras.utils import to_categorical
import tensorflow as tf
import datetime

In [25]:
architecture = { 'stages' : [('Stage1', 3), ('Stage2', 5)], 
                 'code' : np.array([0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1]),
                }
hyper_params = { 'optimizer' : 'adam',
                 'drop out' : 0.5,
                 'epochs' : 20,
                 'kernel size' : (5, 5),
                 'pool size' : 2,
                 'strides' : 2,
                 'filters' : 20,
                 'fc units': 500,
                 'pooling' : 'max'}

In [26]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [27]:
# Convert class vectors to binary class matrices.
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [28]:
# Data normalization
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [29]:
model = genetic_model(architecture, hyper_params,
                     input_shape=x_train.shape[1:], classes=num_classes)

In [14]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [16]:
# Place the logs in a timestamped subdirectory
# This allows to easy select different training runs
# In order not to overwrite some data, it is useful to have a name with a timestamp
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Specify the callback object
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# tf.keras.callback.TensorBoard ensures that logs are created and stored
# We need to pass callback object to the fit method
# The way to do this is by passing the list of callback objects, which is in our case just one

In [30]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 32, 32, 20)   1520        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_9 (BatchNor (None, 32, 32, 20)   80          conv2d_9[0][0]                   
__________________________________________________________________________________________________
activation_9 (Activation)       (None, 32, 32, 20)   0           batch_normalization_9[0][0]      
____________________________________________________________________________________________

In [ ]:
model.fit(x_train, y=y_train, 
          epochs=20, 
          validation_data=(x_test, y_test),
          verbose=1)

Epoch 1/20
1563/1563 [==============================] - 27s 17ms/step - loss: 1.6900 - accuracy: 0.3884 - val_loss: 1.7006 - val_accuracy: 0.4501
Epoch 2/20
1563/1563 [==============================] - 26s 17ms/step - loss: 1.2354 - accuracy: 0.5611 - val_loss: 1.0882 - val_accuracy: 0.6163
Epoch 3/20
1563/1563 [==============================] - 26s 17ms/step - loss: 1.0134 - accuracy: 0.6459 - val_loss: 1.0557 - val_accuracy: 0.6295
Epoch 4/20
1563/1563 [==============================] - 26s 17ms/step - loss: 0.8798 - accuracy: 0.6955 - val_loss: 0.9473 - val_accuracy: 0.6773
Epoch 5/20
1563/1563 [==============================] - 26s 17ms/step - loss: 0.7827 - accuracy: 0.7335 - val_loss: 0.9619 - val_accuracy: 0.6663
Epoch 6/20
1563/1563 [==============================] - 26s 17ms/step - loss: 0.7083 - accuracy: 0.7588 - val_loss: 1.0449 - val_accuracy: 0.6665
Epoch 7/20
1563/1563 [==============================] - 27s 17ms/step - loss: 0.6385 - accuracy: 0.7836 - val_loss: 0.7011 -

In [19]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 5468), started 0:38:11 ago. (Use '!kill 5468' to kill it.)